In [ ]:
import pandas as pd
file_path = '/Users/saifalikhan/Python-Anaconda/Ph.D. Work/Lumpy/Lumpy_data_updated.xlsx'
df = pd.read_excel(file_path)
df.head()

In [ ]:
country_case_counts = df.groupby('Country')['Diagnosis.status'].count().reset_index()

country_case_counts = country_case_counts.rename(columns={'Diagnosis.status': 'Case Count'})

top_12_countries = country_case_counts.sort_values(by='Case Count', ascending=False).head(10)

top_12_countries


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.dates import DateFormatter
from matplotlib.ticker import MaxNLocator

# Filter the dataset for only the top 12 countries
top_countries = top_12_countries['Country'].tolist()
filtered_df = df[df['Country'].isin(top_countries)]

# Convert the 'report_date' column to datetime format for proper time series plotting
filtered_df['Reporting Date'] = pd.to_datetime(filtered_df['Reporting Date'])

# Group by Country and report_date to get the daily case counts
time_series_data = (
    filtered_df.groupby(['Country', 'Reporting Date'])
    .size()
    .reset_index(name='Case Count')
)

# Create subplots (3 rows x 4 columns)
fig, axes = plt.subplots(5, 2, figsize=(20, 12))  # Removed `sharex=True` for independent x-axis ranges
axes = axes.flatten()  # Flatten the 3x4 grid into a 1D array for easy iteration


for idx, country in enumerate(top_countries):
    ax = axes[idx]
    country_data = time_series_data[time_series_data['Country'] == country]
    
    sns.lineplot(
        data=country_data,
        x='Reporting Date',
        y='Case Count',
        ax=ax,
        #color=custom_palette[idx]  # Assign a unique color for each country
    )
    
    ax.xaxis.set_major_formatter(DateFormatter("%Y-%m"))
    ax.xaxis.set_major_locator(MaxNLocator(5))  # Ensure only 5 ticks are shown on the x-axis

    ax.set_title(country, fontsize=14, fontweight = 'bold')
    ax.set_xlabel("Timespan", fontsize=12, fontweight = 'bold')
    ax.set_ylabel("Case Count", fontsize=12, fontweight = 'bold')

for i in range(len(top_countries), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
# Save the combined figure with bbox_to_anchor legend properly included
plt.savefig('trend_lumpy_data_by_country.png', dpi=500)
plt.show()


In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
import pandas as pd

# Create a DataFrame to store ADF test results
adf_results = []

fig, axs = plt.subplots(10, 2, figsize=(15, 15))  # 10x2 grid for ACF and PACF
axs = axs.reshape(10, 2)  # Ensure it is reshaped to (10, 2)

for idx, country in enumerate(top_countries[:10]):  # Only top 10 countries
    country_data = time_series_data[time_series_data['Country'] == country]
    
    daily_cases = country_data.groupby('Reporting Date')['Case Count'].sum()
    
    # ADF test results
    adf_result = adfuller(daily_cases)
    stationary = adf_result[1] < 0.05  # If p-value < 0.05, the series is stationary
    
    adf_results.append({
        'Country': country,
        'ADF Statistic': adf_result[0],
        'p-value': adf_result[1],
        'Stationary': 'Yes' if stationary else 'No'
    })
    
    max_lags = max(1, len(daily_cases) // 2 - 1)  # Ensure at least 1 lag and up to half of the data points
    
    plot_acf(daily_cases, ax=axs[idx, 0], lags=max_lags)
    plot_pacf(daily_cases, ax=axs[idx, 1], lags=max_lags)
    
    axs[idx, 0].set_title(f'{country} ACF', fontsize=12, fontweight = 'bold')
    axs[idx, 1].set_title(f'{country} PACF', fontsize=12, fontweight = 'bold')

fig.tight_layout()

adf_results_df = pd.DataFrame(adf_results)
plt.savefig('ACF_PACF.png', dpi=500)

plt.show()


# Model Comparison

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pmdarima import auto_arima
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from matplotlib.ticker import MaxNLocator

def train_predict_autoarima(train_data, test_data):
    model = auto_arima(train_data, seasonal=True, suppress_warnings=True, stepwise=True, error_action='ignore', max_d=2)
    forecast = model.predict(n_periods=len(test_data))
    mse = mean_squared_error(test_data, forecast)
    return forecast, mse

def train_predict_decision_tree(X_train, y_train, X_test):
    model = DecisionTreeRegressor()
    model.fit(X_train, y_train)
    forecast = model.predict(X_test)
    mse = mean_squared_error(y_test, forecast)  # Assuming y_test is available globally
    return forecast, mse

def train_predict_random_forest(X_train, y_train, X_test):
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    forecast = model.predict(X_test)
    mse = mean_squared_error(y_test, forecast)  # Assuming y_test is available globally
    return forecast, mse

def train_predict_svr(X_train, y_train, X_test):
    model = SVR(kernel='rbf', C=100, gamma='scale')
    model.fit(X_train, y_train)
    forecast = model.predict(X_test)
    mse = mean_squared_error(y_test, forecast)  # Assuming y_test is available globally
    return forecast, mse

def plot_results(ax, train_data, test_data, forecast, title):
    combined_data = pd.concat([train_data, test_data])
    
    ax.plot(combined_data.index, combined_data.values, label='Actual', color='blue', linestyle='--')
    
    ax.plot(train_data.index, train_data.values, label='Train', color='blue', linestyle='--')
    ax.plot(test_data.index, test_data.values, label='Test', color='orange', linestyle='--')
    
    ax.plot(test_data.index, forecast, label='Forecast', color='green')
    
    ax.set_title(title, fontweight='bold')
    ax.set_xlabel('Date', fontweight='bold')
    ax.set_ylabel('Lumpy Cases', fontweight='bold')
    ax.xaxis.set_major_locator(MaxNLocator(nbins=4))
    ax.tick_params(axis='x', rotation=0)
    return ax

df = pd.read_excel('/Users/saifalikhan/Python-Anaconda/Ph.D. Work/Lumpy/Lumpy_data_updated.xlsx', parse_dates=['Reporting Date'])

top_countries = df.groupby('Country')['Diagnosis.status'].sum().nlargest(10).index

combined_results = []

fig, axs = plt.subplots(10, 4, figsize=(20, 18), dpi=500)
axs = axs.flatten()

models = ['AutoARIMA', 'Decision Tree', 'Random Forest', 'SVR']
model_functions = [train_predict_autoarima, train_predict_decision_tree, train_predict_random_forest, train_predict_svr]

train_lines = []
test_lines = []
forecast_lines = []

for i, country in enumerate(top_countries):
    country_data = df[df['Country'] == country]
    country_data = country_data.sort_values('Reporting Date')
    
    daily_cases = country_data.groupby('Reporting Date')['Diagnosis.status'].sum()
    
    lags = 5
    data = pd.DataFrame({'y': daily_cases})
    for lag in range(1, lags + 1):
        data[f'lag_{lag}'] = data['y'].shift(lag)
    data = data.dropna()
    
    X = data.drop(columns=['y'])
    y = data['y']
    train_size = int(0.8 * len(data))
    X_train, X_test, y_train, y_test = X.iloc[:train_size], X.iloc[train_size:], y.iloc[:train_size], y.iloc[train_size:]
    
    try:
        for j, (model_name, model_func) in enumerate(zip(models, model_functions)):
            if model_name == 'AutoARIMA':
                forecast, mse = model_func(y_train, y_test)
            else:
                forecast, mse = model_func(X_train, y_train, X_test)
            
            plot_results(axs[i * len(models) + j], y_train, y_test, forecast, f'{country} - {model_name}')
            
            combined_results.append({
                'Country': country,
                'Model': model_name,
                'MSE': mse,
                'Forecast': forecast.tolist()
            })
            
            # Collect lines for legend
            if i == 0 and j == 0:
                train_lines.append(axs[i * len(models) + j].lines[1])  # Train line
                test_lines.append(axs[i * len(models) + j].lines[2])   # Test line
                forecast_lines.append(axs[i * len(models) + j].lines[3])  # Forecast line
            
    except Exception as e:
        print(f"Error occurred while processing {country}: {e}")

plt.tight_layout()

fig.legend(handles=[train_lines[0], test_lines[0], forecast_lines[0]], labels=['Train', 'Test', 'Forecast'],
           loc='upper right', bbox_to_anchor=(1.04, 0.98), fontsize='large')


combined_results_df = pd.DataFrame(combined_results)
#combined_results_df.to_excel("combined_results.xlsx", index=False)

plt.show()


In [ ]:
combined_results_df

# Cross validation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pmdarima import auto_arima
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from matplotlib.ticker import MaxNLocator

# AutoARIMA forecast
def train_predict_autoarima(train_data, test_data):
    model = auto_arima(train_data, seasonal=True, suppress_warnings=True, stepwise=True, error_action='ignore', max_d=2)
    forecast = model.predict(n_periods=len(test_data))
    mse = mean_squared_error(test_data, forecast)
    return forecast, mse

# Decision Tree with fixed hyperparameters
def train_predict_decision_tree(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [3, 5, 10, None],
        'min_samples_leaf': [1, 2, 4, 6]
    }

    tscv = TimeSeriesSplit(n_splits=5)
    dt = DecisionTreeRegressor(random_state=42)
    grid_search = GridSearchCV(dt, param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    print(f"Best Decision Tree params: {grid_search.best_params_}")

    forecast = best_model.predict(X_test)
    mse = mean_squared_error(y_test, forecast)
    return forecast, mse


# Random Forest with hyperparameter tuning using time series CV
def train_predict_random_forest(X_train, y_train, X_test, y_test):
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 10, None],
        'min_samples_leaf': [1, 2, 4]
    }

    tscv = TimeSeriesSplit(n_splits=5)
    rf = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(rf, param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    
    print(f"Best Random Forest params: {grid_search.best_params_}")
    
    forecast = best_model.predict(X_test)
    mse = mean_squared_error(y_test, forecast)
    return forecast, mse

# SVR
def train_predict_svr(X_train, y_train, X_test, y_test):
    model = SVR(kernel='rbf', C=100, gamma='scale')
    model.fit(X_train, y_train)
    forecast = model.predict(X_test)
    mse = mean_squared_error(y_test, forecast)
    return forecast, mse

# Plotting
def plot_results(ax, train_data, test_data, forecast, title):
    combined_data = pd.concat([train_data, test_data])
    ax.plot(combined_data.index, combined_data.values, label='Actual', color='blue', linestyle='--')
    ax.plot(train_data.index, train_data.values, label='Train', color='blue', linestyle='--')
    ax.plot(test_data.index, test_data.values, label='Test', color='orange', linestyle='--')
    ax.plot(test_data.index, forecast, label='Forecast', color='green')
    ax.set_title(title, fontweight='bold')
    ax.set_xlabel('Date', fontweight='bold')
    ax.set_ylabel('Lumpy Cases', fontweight='bold')
    ax.xaxis.set_major_locator(MaxNLocator(nbins=4))
    ax.tick_params(axis='x', rotation=0)
    return ax

# Load data
df = pd.read_excel('/Users/saifalikhan/Python-Anaconda/Ph.D. Work/Lumpy/Lumpy_data_updated.xlsx', parse_dates=['Reporting Date'])

# Get top 10 countries by cases
top_countries = df.groupby('Country')['Diagnosis.status'].sum().nlargest(10).index

# Results and figure
combined_results = []
fig, axs = plt.subplots(10, 4, figsize=(20, 18), dpi=500)
axs = axs.flatten()

models = ['AutoARIMA', 'Decision Tree', 'Random Forest', 'SVR']
model_functions = [
    lambda yt, yv, *_: train_predict_autoarima(yt, yv),
    train_predict_decision_tree,
    train_predict_random_forest,
    train_predict_svr
]

train_lines, test_lines, forecast_lines = [], [], []

for i, country in enumerate(top_countries):
    country_data = df[df['Country'] == country].sort_values('Reporting Date')
    daily_cases = country_data.groupby('Reporting Date')['Diagnosis.status'].sum()
    
    lags = 5
    data = pd.DataFrame({'y': daily_cases})
    for lag in range(1, lags + 1):
        data[f'lag_{lag}'] = data['y'].shift(lag)
    data = data.dropna()

    X = data.drop(columns=['y'])
    y = data['y']
    train_size = int(0.8 * len(data))
    X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
    y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

    try:
        for j, (model_name, model_func) in enumerate(zip(models, model_functions)):
            if model_name == 'AutoARIMA':
                forecast, mse = model_func(y_train, y_test)
            else:
                forecast, mse = model_func(X_train, y_train, X_test, y_test)
            
            plot_results(axs[i * len(models) + j], y_train, y_test, forecast, f'{country} - {model_name}')
            combined_results.append({
                'Country': country,
                'Model': model_name,
                'MSE': mse,
                'Forecast': forecast.tolist()
            })

            if i == 0 and j == 0:
                train_lines.append(axs[i * len(models) + j].lines[1])
                test_lines.append(axs[i * len(models) + j].lines[2])
                forecast_lines.append(axs[i * len(models) + j].lines[3])

    except Exception as e:
        print(f"Error processing {country} with {model_name}: {e}")

plt.tight_layout()
fig.legend(handles=[train_lines[0], test_lines[0], forecast_lines[0]], labels=['Train', 'Test', 'Forecast'],
           loc='upper right', bbox_to_anchor=(1.04, 0.98), fontsize='large')

# Save if needed
#plt.savefig('combined_forecast_plots.png', bbox_inches='tight', dpi=500)
#plt.savefig('combined_forecast_plots.pdf', bbox_inches='tight', dpi=500)

combined_results_df = pd.DataFrame(combined_results)
#combined_results_df.to_excel("combined_results.xlsx", index=False)

plt.show()


In [ ]:
combined_results